## Как сопоставить время/индекс

Нужно как-то резать промпты, чтобы в контекст влезало

Нужно что-то почитать для промт инжиниринга


В крайний случай нужно будет брать эмбединги из whisper и vit и учить свою модельку на них

C GPT есть проблемы с воспроизведением

In [144]:
import os


import cv2
import torch
import openai
import pandas as pd
import numpy as np
import whisper
from PIL import Image
from moviepy.editor import VideoFileClip
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from sklearn.metrics import f1_score


openai.api_key = os.getenv("OPENAI_APIKEY")

vit_model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
vit_feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
vit_tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

asr_model = whisper.load_model("tiny.en")


FPS = 1 
IMAGE_SIZE = (224, 224)
video_path = "/media/mark/ADATA HV620S/diplom/tvsum/ydata-tvsum50-v1_1/video/Bhxk-O1Y7Ho.mp4"

labels_df = pd.read_parquet('labels.parquet')
labels_df = labels_df.reset_index().drop(columns=['index'])


def save_audio(filepath):
    filename = os.path.basename(filepath).split(".")[0]
    result_path = filename + ".mp3"
    VideoFileClip(filepath).audio.write_audiofile(result_path)
    return result_path
    

def get_images(video_path, fps, image_size):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    images = []

    for i in range(0, frame_count, int(cap.get(cv2.CAP_PROP_FPS) / fps)):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).resize(image_size)
            images.append(image)

    cap.release()
    return images



def get_image_caption(images, feature_extractor, tokenizer, model, **kwags):
    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    output_ids = model.generate(pixel_values, **kwags)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    
    df = pd.DataFrame({"text": [pred.strip() for pred in preds]})
    df = df.reset_index().rename(columns={"index": "second"})
    return df


def get_text_recognition(asr_model, audio_file):
    asr_result = asr_model.transcribe(audio_file)
    return pd.DataFrame([
        {
            'start_second': segment['start'],
            'end_second': segment['end'],
            'text': segment['text'],
        }
        for segment in asr_result['segments']
    ])


def create_prompt(asr_result, ic_result):
    return """
        Summarize the video by description data
        Decription of each image per second:
        {ic_result}

        Speech recognition results:
        {asr_result}
        Return summarization as table with columns of second and importance. importance should be from 0 to 1
        """.format(ic_result=ic_result.to_string(index=None),
                   asr_result=asr_result.to_string(index=None)
                  )


def transform_predictions(pred_text):
    result = []
    for line in pred_text.split('\n'):
        pnumbers = line.strip().split() 
        if len(pnumbers) != 2:
            continue
        try:
            result.append({'second': float(pnumbers[0]), 'prob': float(pnumbers[1])})
        except:
            continue
    return pd.DataFrame(result)


def fill_preds(ic_result, preds):
    sec = np.array(range(ic_result['second'].max() + 1))
    prob = np.zeros(shape=sec.shape)
    
    
    for i in range(preds.shape[0]-1):
        prob[(sec >= preds.iloc[i].second) & (sec <= preds.iloc[i+1].second)] = preds.iloc[i].prob
    last_i = preds.shape[0]-1
    prob[sec >= preds.iloc[last_i].second] = preds.iloc[last_i].prob  
    return pd.DataFrame({'second': sec, 'prob': prob})


def eval_metric(preds, annotation, t=0.5):
    post_preds = preds.copy()
    post_preds['reps'] = annotation.shape[0] // preds['second'].max()
    post_preds = post_preds.loc[post_preds.index.repeat(post_preds.reps)]
    post_preds = post_preds['prob'].values
    
    same_len = min(post_preds.shape[0], annotation.shape[0])
    y_true = np.where(annotation[:same_len] >= t, 1, 0)
    y_pred = np.where(post_preds[:same_len] >= t, 1, 0)
    return f1_score(y_true, y_pred)

In [145]:
video_path = "/media/mark/ADATA HV620S/diplom/tvsum/ydata-tvsum50-v1_1/video/0tmA_C6XwfM.mp4"
ic_kwargs = {"max_length": 10, "num_beams": 2}


audio_path = save_audio(video_path)
images = get_images(video_path, FPS, IMAGE_SIZE)
ic_result = get_image_caption(images, vit_feature_extractor, vit_tokenizer, vit_model, **ic_kwargs)

asr_result = get_text_recognition(asr_model, '0tmA_C6XwfM.mp3')
prompt = create_prompt(asr_result, ic_result)

response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0.7,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response['choices'][0]['text'])
tdf = transform_predictions(response['choices'][0]['text'])
preds = fill_preds(ic_result, tdf)

print("Sample of preds", preds)
filename = os.path.basename(video_path).split(".")[0]
annotation = labels_df[labels_df['video_id'] == filename].annotation.values[0]

eval_metric(preds, annotation)

MoviePy - Writing audio in 0tmA_C6XwfM.mp3


MoviePy - Done.

        second       importance
        0            0.8
        12.96        0.7
        18.76        0.6
        23.96        0.5
        29.88        0.6
        34.64        0.5
        37.64        0.5
        42.36        0.4
        46.08        0.4
        51.40        0.4
        54.92        0.3
        56.68        0.3
        60.56        0.3
        65.00        0.4
        74.40        0.4
        79.44        0.3
        81.24        0.3
        87.80        0.4
        91.44        0.4
        97.44        0.3
        102.44       0.3
        107.48       0.3
        111.24       0.3
        115.80       0.2
        117.44       0.2
        122.68       0.3
        127.32       0.3
        129.00       0.3

Sample of preds      second  prob
0         0   0.8
1         1   0.8
2         2   0.8
3         3   0.8
4         4   0.8
..      ...   ...
137     137   0.3
138     138   0.3
139     139   0.3
140     140   0.3
141     141   0.3

[142 rows x 2 col

0.05194805194805195